In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import numpy as np
import pandas as pd

import copy
import traceback
import datetime
import joblib
import re
import os
import random
import string
import time

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import matplotlib.pyplot as plt
%matplotlib inline

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from torchtext import data
from torchtext import datasets
from torchtext import vocab
from torchtext.vocab import Vectors, GloVe

from tqdm.notebook import tqdm

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
import pymorphy2
from sklearn.base import BaseEstimator, TransformerMixin

In [3]:
class ConfigExperiment:
    seed = 42
    positive_file = "../data/positive.csv"
    negative_file = "../data/negative.csv"
    test_size = 0.3
    device = "cuda" if torch.cuda.is_available() else "cpu"
    embed_dim = 300
    max_vocab_size = 50_000
    batch_size = 64
    num_epochs = 30
    lr = 1e-2
    num_workers = 0
    patience = 3
    early_stopping_delta = 1e-4
    save_dirname = "models"
    
config = ConfigExperiment()

In [4]:
def init_random_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic=True
    
init_random_seed(config.seed)

In [5]:
column_names = ["id", "tdate", "tmane", "ttext", "ttype", "trep", "trtw", "tfav", "tstcount", "tfoll", "tfrien", "listcount"]
positive_df = pd.read_csv(config.positive_file, sep=";", names=column_names, index_col=False)
negative_df = pd.read_csv(config.negative_file, sep=";", names=column_names, index_col=False)

# Смена метки класса для отрицательной эмоциональной окраски
negative_df["ttype"] = 0

df = pd.concat([negative_df, positive_df])
df = df[["ttext", "ttype"]]
df.columns = ['text', 'target']

df.shape, negative_df.shape, positive_df.shape

((226834, 2), (111923, 12), (114911, 12))

In [6]:
df["text"].tolist()[:40]

['на работе был полный пиддес :| и так каждое закрытие месяца, я же свихнусь так D:',
 'Коллеги сидят рубятся в Urban terror, а я из-за долбанной винды не могу :(',
 '@elina_4post как говорят обещаного три года ждут...((',
 'Желаю хорошего полёта и удачной посадки,я буду очень сильно скучать( http://t.co/jCLNzVNv3S',
 'Обновил за каким-то лешим surf, теперь не работает простоплеер :(',
 'Котёнка вчера носик разбила, плакала и расстраивалась :(',
 '@juliamayko @O_nika55 @and_Possum Зашли, а то он опять затихарился, я прямо физически страдаю, когда он долго молчит!(((',
 'а вообще я не болею -  я не выздоравливаю :(',
 'я микрофраза :( учимся срать кирпичами в режиме &amp;quot;нон-стоп&amp;quot; @niwoqisipapy',
 'я хочу с тобой помириться , но сука я гордая и никогда этого не сделаю! (((',
 '@DNO_OKEANA_A3A3 @MOE_MOPE_A3A3 тебя ебет какие у меня фотки.я про твои молчу.и вообще ты хоть знаешь как ТП то выглядят...',
 'Блин начали сниться сны( Не когда почти не снились ! А теперь можно ска

In [7]:
def preprocess_text(text):
    text = text.lower().replace("ё", "е")
    # Remove digits
    text = re.sub("\d+:\d+", " ", text)
    text = re.sub(" \d+", " ", text)
    # Removing ';quot;'
    text = re.sub(';quot;', '', text) 
    # Removing ';quot;'
    text = re.sub('&amp', '', text) 
    # Remove HTML special entities (e.g. &amp;)
    text = re.sub(r'\&\w*;', ' ', text)
    #Convert @username to AT_USER
    text = re.sub('@[^\s]+','AT_USER', text)
    #Removing @mentions
    text = re.sub('@[A-Za-z0–9]+', '', text)
    # Remove whitespace (including new line characters)
    text = re.sub(r'\s\s+', ' ', text)
    # Removing '#' hash tag
    text = re.sub('#', '', text) 
    # Removing RT
    text = re.sub('rt[\s]+', '', text) 
    # Removing hyperlink
    text = re.sub('https?:\/\/\S+', '', text)
    # Separate words and punctuation
    text = re.findall(r"[\w']+|[.,!?;:()]", text)
    text = " ".join(text)
    return text

In [8]:
# Clean the tweets
df['text'] = df['text'].apply(preprocess_text)

In [9]:
df["text"].tolist()[:40]

['на работе был полный пиддес : и так каждое закрытие месяца , я же свихнусь так d :',
 'коллеги сидят рубятся в urban terror , а я из за долбанной винды не могу : (',
 'AT_USER как говорят обещаного три года ждут . . . ( (',
 'желаю хорошего полета и удачной посадки , я буду очень сильно скучать (',
 'обновил за каким то лешим surf , теперь не работает простоплеер : (',
 'котенка вчера носик разбила , плакала и расстраивалась : (',
 'AT_USER AT_USER AT_USER зашли , а то он опять затихарился , я прямо физически страдаю , когда он долго молчит ! ( ( (',
 'а вообще я не болею я не выздоравливаю : (',
 'я микрофраза : ( учимся срать кирпичами в режиме нон стоп AT_USER',
 'я хочу с тобой помириться , но сука я гордая и никогда этого не сделаю ! ( ( (',
 'AT_USER AT_USER тебя ебет какие у меня фотки . я про твои молчу . и вообще ты хоть знаешь как тп то выглядят . . .',
 'блин начали сниться сны ( не когда почти не снились ! а теперь можно сказать каждый день такое ! мне это не нравиться ( 

In [10]:
cache = {}
morph = pymorphy2.MorphAnalyzer()

# Фильтруем по части речи и возвращаем только начальную форму.
def lemmatize(text):
    words = []
    for token in text.split():
        # Если токен уже был закеширован, быстро возьмем результат из кэша.
        if token in cache.keys():
            words.append(cache[token])
        # Слово еще не встретилось, будем проводить медленный морфологический анализ.
        else:
            result = morph.parse(token)   
            word = result[0].normal_form
            # Отправляем слово в результат, ...
            words.append(word)
            # ... и кешируем результат его разбора.
            cache[token] = word   
    return ' '.join(words)

In [11]:
%%time

df['text'] = df['text'].apply(lemmatize)

CPU times: user 40.3 s, sys: 110 ms, total: 40.4 s
Wall time: 40.4 s


In [12]:
df["text"].tolist()[:40]

['на работа быть полный пиддес : и так каждый закрытие месяц , я же свихнуться так d :',
 'коллега сидеть рубиться в urban terror , а я из за долбать винд не мочь : (',
 'at_user как говорят обещаной три год ждать . . . ( (',
 'желать хороший полёт и удачный посадка , я быть очень сильно скучать (',
 'обновить за какой то леший surf , теперь не работать простоплеер : (',
 'котёнок вчера носик разбить , плакать и расстраиваться : (',
 'at_user at_user at_user заслать , а то он опять затихариться , я прямо физически страдать , когда он долго молчать ! ( ( (',
 'а вообще я не болеть я не выздоравливать : (',
 'я микрофраза : ( учиться срать кирпич в режим нона стоп at_user',
 'я хотеть с ты помириться , но сук я гордый и никогда это не сделать ! ( ( (',
 'at_user at_user ты ебета какой у я фотка . я про твой молчать . и вообще ты хоть знаешь как тп то выглядеть . . .',
 'блин начать сниться сон ( не когда почти не сниться ! а теперь можно сказать каждый день такой ! я это не нравиться ( (

In [13]:
# Формирование train test valid данных

# df = pd.read_csv("../data/preprocessed_text_v1.csv", index_col=False)

df = df.drop(df[df['text'].map(str) == 'nan'].index)

train, validate, test = np.split(df.sample(frac=1), [int(.6*len(df)), int(.8*len(df))])
train.to_csv("../data/train_processed_data.csv", index=False)
validate.to_csv("../data/validate_processed_data.csv", index=False)
test.to_csv("../data/test_processed_data.csv", index=False)

train.shape, validate.shape, test.shape

((136100, 2), (45367, 2), (45367, 2))

In [14]:
tokenize = lambda x: str(x).split()

TEXT = data.Field(sequential=True, tokenize=tokenize, batch_first=True)
LABEL = data.LabelField(dtype=torch.float)

fields = [('text',TEXT), ('label', LABEL)]

In [15]:

train_data, valid_data, test_data = data.TabularDataset.splits(
                                        path="../data/",
                                        train="train_processed_data.csv",
                                        validation="validate_processed_data.csv",
                                        test="test_processed_data.csv",
                                        format="csv",
                                        fields=fields,
                                        skip_header=True
)

In [16]:
print(vars(train_data[0]))

{'text': ['оставаться', 'самый', 'нужный', 'и', 'самый', 'близкие', ')', 'весь', 'остальной', 'уходить', ')', 'и', 'я', 'только', 'рада', ')', 'потому', 'что', 'я', 'никогда', 'сам', 'не', 'понять', 'нужный', 'я', 'человек', 'или', 'нет', ')'], 'label': '1'}


In [17]:
print(vars(valid_data[0]))

{'text': ['at_user', 'at_user', 'я', 'старушка', '(', '(', '('], 'label': '0'}


In [18]:
print(vars(test_data[0]))

{'text': ['at_user', 'привееть', ',', 'хелена', ':', ')', ')', ')', 'мимими', '.', '.', '.', 'пряник', ',', 'конфета', 'и', 'сирец'], 'label': '1'}


In [19]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 136100
Number of validation examples: 45367
Number of testing examples: 45367


In [20]:
# TEXT.build_vocab(train_data, max_size=config.max_vocab_size)
TEXT.build_vocab(train_data, min_freq=2)
LABEL.build_vocab(train_data)

In [21]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 33015
Unique tokens in LABEL vocabulary: 2


In [22]:
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    sort_key = lambda x: x.text,
    batch_size=config.batch_size,
    device=config.device)

In [23]:
# print('Train:')
# for batch in train_iterator:
#     print(batch)
    
# print('Valid:')
# for batch in valid_iterator:
#     print(batch)
    
# print('Test:')
# for batch in test_iterator:
#     print(batch)

In [24]:
print(TEXT.vocab.freqs.most_common(20))

[('(', 126857), (')', 116605), (',', 112838), ('.', 110646), ('at_user', 89459), (':', 64837), ('я', 64791), ('не', 44975), ('!', 39989), ('и', 36467), ('в', 35466), ('что', 23049), ('на', 22897), ('а', 21841), ('?', 21683), ('с', 20052), ('весь', 18848), ('ты', 18175), ('быть', 16970), ('это', 14896)]


In [25]:
print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', '(', ')', ',', '.', 'at_user', ':', 'я', 'не']


In [26]:
print(LABEL.vocab.stoi)

defaultdict(None, {'1': 0, '0': 1})


In [27]:
import zipfile
import gensim
import wget

model_url = 'http://vectors.nlpl.eu/repository/11/187.zip'
# wget.download(model_url)
w2v_model = gensim.models.KeyedVectors.load('187/model.model')
numpy_embeddings = np.zeros(shape=[len(TEXT.vocab), config.embed_dim],dtype=np.float32)

for word in TEXT.vocab.itos:
    vector = w2v_model.get_vector(word)
    index  = TEXT.vocab.stoi[word]
    numpy_embeddings[index] = vector
    
pretrained_embeddings = torch.Tensor(numpy_embeddings).float()
pretrained_embeddings.shape

torch.Size([33015, 300])

In [28]:
class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, dropout, pad_idx):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        self.convs = nn.ModuleList([
                                    nn.Conv2d(in_channels = 1, 
                                              out_channels = n_filters, 
                                              kernel_size = (fs, embedding_dim)) 
                                    for fs in filter_sizes
                                    ])
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        #text = [batch size, sent len]
        embedded = self.embedding(text)
        #embedded = [batch size, sent len, emb dim]
        embedded = embedded.unsqueeze(1)
        #embedded = [batch size, 1, sent len, emb dim]
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
        #conved_n = [batch size, n_filters, sent len - filter_sizes[n] + 1]
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        #pooled_n = [batch size, n_filters]
        cat = self.dropout(torch.cat(pooled, dim = 1))
        #cat = [batch size, n_filters * len(filter_sizes)]
        return self.fc(cat)

In [29]:
class CNN1d(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, 
                 dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.convs = nn.ModuleList([
                                    nn.Conv1d(in_channels = embedding_dim, 
                                              out_channels = n_filters, 
                                              kernel_size = fs)
                                    for fs in filter_sizes
                                    ])
        
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        #text = [batch size, sent len]
        
        embedded = self.embedding(text)
                
        #embedded = [batch size, sent len, emb dim]
        
        embedded = embedded.permute(0, 2, 1)
        
        #embedded = [batch size, emb dim, sent len]
        
        conved = [F.relu(conv(embedded)) for conv in self.convs]
            
        #conved_n = [batch size, n_filters, sent len - filter_sizes[n] + 1]
        
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        
        #pooled_n = [batch size, n_filters]
        
        cat = self.dropout(torch.cat(pooled, dim = 1))
        
        #cat = [batch size, n_filters * len(filter_sizes)]
            
        return self.fc(cat)

In [30]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = config.embed_dim
N_FILTERS = 128
FILTER_SIZES = [2, 3]
OUTPUT_DIM = 1
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = CNN1d(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)

In [31]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 10,097,013 trainable parameters


In [32]:
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[ 3.0550e-02, -3.5227e-01, -1.5996e-01,  ..., -6.7555e-01,
          1.6213e-01, -6.4838e-01],
        [ 5.5938e-01,  7.1610e-01, -5.2220e-02,  ..., -3.1100e-01,
          1.0994e+00, -7.4458e-01],
        [ 8.6657e-04,  2.8588e-03, -1.2546e-03,  ...,  2.7372e-03,
          3.3164e-03,  2.3352e-03],
        ...,
        [ 1.2707e-01, -2.0811e-02,  1.6323e-01,  ..., -1.2492e-03,
          2.0801e-01, -2.6840e-03],
        [-1.0621e+00,  2.0553e+00, -1.1423e+00,  ...,  3.4679e-01,
         -3.4598e-01, -1.1714e+00],
        [-2.3426e-02,  7.1213e-02, -9.7777e-02,  ...,  1.2438e-04,
         -1.1894e-03,  2.1802e-02]])

In [33]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

In [34]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

criterion = nn.BCEWithLogitsLoss()

model = model.to(config.device)
criterion = criterion.to(config.device)

In [35]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [36]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [37]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [38]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [39]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut4-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 20s
	Train Loss: 0.124 | Train Acc: 94.01%
	 Val. Loss: 0.017 |  Val. Acc: 99.24%
Epoch: 02 | Epoch Time: 0m 20s
	Train Loss: 0.020 | Train Acc: 99.24%
	 Val. Loss: 0.016 |  Val. Acc: 99.34%
Epoch: 03 | Epoch Time: 0m 20s
	Train Loss: 0.018 | Train Acc: 99.35%
	 Val. Loss: 0.017 |  Val. Acc: 99.32%
Epoch: 04 | Epoch Time: 0m 22s
	Train Loss: 0.015 | Train Acc: 99.49%
	 Val. Loss: 0.025 |  Val. Acc: 99.24%
Epoch: 05 | Epoch Time: 0m 21s
	Train Loss: 0.014 | Train Acc: 99.55%
	 Val. Loss: 0.024 |  Val. Acc: 99.32%


In [40]:
model.load_state_dict(torch.load('tut4-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.016 | Test Acc: 99.34%


In [41]:
# for CNN2d 
# Test Loss: 0.371 | Test Acc: 84.98%

In [42]:
# for CNN1d 
# Test Loss: 0.333 | Test Acc: 85.73%